In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from urllib.parse import urlparse
import time
import os
import numpy as np
import math
from datetime import datetime, timedelta
# import holidays
import warnings
import argparse

In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()

# You can modify the following to use a bucket of your choosing
bucket = session.default_bucket()
prefix = "sagemaker/autopilot-water-demand-prediction"

role = get_execution_role()

# This is the client we will use to interact with SageMaker Autopilot
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
from io import StringIO

s3 = boto3.client("s3")

def list_csv_files(bucket_name, key_path):
    # List objects within the specified bucket and prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=key_path)
    # Filter out the CSV files
    csv_files = [content['Key'] for content in response.get('Contents', []) if content['Key'].endswith('.csv')]
    return csv_files

def read_csv_files_to_dataframes(bucket_name, csv_files):
    dataframes = []
    for key in csv_files:
        # Get the object from S3
        obj = s3.get_object(Bucket=bucket_name, Key=key)
        # Read the CSV file content
        data = obj['Body'].read().decode('utf-8')
        # Convert to DataFrame
        df = pd.read_csv(StringIO(data))
        dataframes.append(df)
    return dataframes

# Example usage
bucket_name = 'niwa-water-demand-modelling'
key_path_rf = 'WWL/bias_adjusted_data/'
key_path_temp = 'WWL/cmip6/'
csv_files_rf = list_csv_files(bucket_name, key_path_rf)
csv_files_temp = list_csv_files(bucket_name, key_path_temp)

In [4]:
csv_files_temp

['WWL/cmip6/tas_ACCESS-CM2_hist_BirchLane.csv',
 'WWL/cmip6/tas_ACCESS-CM2_hist_PinehavenStream.csv',
 'WWL/cmip6/tas_ACCESS-CM2_ssp_BirchLane.csv',
 'WWL/cmip6/tas_ACCESS-CM2_ssp_PinehavenStream.csv',
 'WWL/cmip6/tas_AWI-CM-1-1-MR_hist_BirchLane.csv',
 'WWL/cmip6/tas_AWI-CM-1-1-MR_hist_PinehavenStream.csv',
 'WWL/cmip6/tas_AWI-CM-1-1-MR_ssp_BirchLane.csv',
 'WWL/cmip6/tas_AWI-CM-1-1-MR_ssp_PinehavenStream.csv',
 'WWL/cmip6/tas_CNRM-CM6-1_hist_BirchLane.csv',
 'WWL/cmip6/tas_CNRM-CM6-1_hist_PinehavenStream.csv',
 'WWL/cmip6/tas_CNRM-CM6-1_ssp_BirchLane.csv',
 'WWL/cmip6/tas_CNRM-CM6-1_ssp_PinehavenStream.csv',
 'WWL/cmip6/tas_EC-Earth3_hist_BirchLane.csv',
 'WWL/cmip6/tas_EC-Earth3_hist_PinehavenStream.csv',
 'WWL/cmip6/tas_EC-Earth3_ssp_BirchLane.csv',
 'WWL/cmip6/tas_EC-Earth3_ssp_PinehavenStream.csv',
 'WWL/cmip6/tas_GFDL-ESM4_hist_BirchLane.csv',
 'WWL/cmip6/tas_GFDL-ESM4_hist_PinehavenStream.csv',
 'WWL/cmip6/tas_GFDL-ESM4_ssp_BirchLane.csv',
 'WWL/cmip6/tas_GFDL-ESM4_ssp_Pinehave

In [5]:
csv_files_rf

['WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_hist_adjusted_1960_2014.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_ssp126_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_ssp245_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_ssp370_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_ssp585_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Pinehaven_ACCESS-CM2_hist_adjusted_1960_2014.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Pinehaven_ACCESS-CM2_ssp126_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Pinehaven_ACCESS-CM2_ssp245_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Pinehaven_ACCESS-CM2_ssp370_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Pinehaven_ACCESS-CM2_ssp585_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/AWI-CM-1-1-MR/Birch Lane_AWI-CM-1-1-MR_hist_adjusted_1960_2014.csv',
 'WWL/bias_adju

In [6]:
def prep_rainfall(df):
    '''
    preprocess rainfall data
    '''
    df['hour'] = df['Time'].dt.hour
    df['wday'] = df['Time'].dt.dayofweek
    df['month'] = df['Time'].dt.month
    df['mday'] = df['Time'].dt.days_in_month
    df["doy"] = df['Time'].dt.dayofyear
    df['Rainlag1'] = df['Rainfall'].shift(1)
    df['Rainlag2'] = df['Rainfall'].shift(2)
    df['Rainlag1'] = df['Rainlag1'].bfill()
    df['Rainlag2'] = df['Rainlag2'].bfill()
    df['Rain_L3HR'] = df.loc[:,'Rainfall'].rolling(window=3).sum()
    df['Rain_L6HR'] = df.loc[:,'Rainfall'].rolling(window=6).sum()
    df['Rain_L12HR'] = df.loc[:,'Rainfall'].rolling(window=12).sum()
    df['Rain_L24HR'] = df.loc[:,'Rainfall'].rolling(window=24).sum()
    df['Rain_L48HR'] = df.loc[:,'Rainfall'].rolling(window=48).sum()
    df['Rain_L3HR'] = df['Rain_L3HR'].bfill()
    df['Rain_L6HR'] = df['Rain_L6HR'].bfill()
    df['Rain_L12HR'] = df['Rain_L12HR'].bfill()
    df['Rain_L24HR'] = df['Rain_L24HR'].bfill()
    df['Rain_L48HR'] = df['Rain_L48HR'].bfill()
    
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)
    
    # Typical value between 0.8 and 0.95
    k_api = 0.85
    
    # Initialize API and Soil Moisture as columns
    df['API'] = 0
    df['SoilMoisture'] = 0
    
    # Calculate Antecedent Precipitation Index (API)
    for i in range(1, len(df)):
        df.at[i, 'API'] = k_api * df.at[i - 1, 'API'] + df.at[i - 1, 'Rainfall']
    
    # Soil Moisture Index (very simplified: saturates at some threshold)
    # Arbitrary max bucket size (mm)
    max_storage = 100
    # Moisture decay rate
    soil_decay = 0.95
    # Initial condition
    df.at[0, 'SoilMoisture'] = min(max_storage, df.at[0, 'Rainfall'])
    
    for i in range(1, len(df)):
        df.at[i, 'SoilMoisture'] = min(
            max_storage,
            df.at[i - 1, 'SoilMoisture'] * soil_decay + df.at[i - 1, 'Rainfall']
        )

    return df

In [7]:
# Extract exp_name and rf_name from each string
temp_extracts = []
for path in csv_files_temp:
    x = path.split('.csv')[0]
    exp_name = '_'.join(x.split('/')[-1].split('_')[1:3])  # model + scenario
    rf_name = x.split('/')[-1].split('_')[-1] # location
    rf_name = rf_name.replace('Stream', '')
    print({'exp_name': exp_name, 'rf_name': rf_name})
    temp_extracts.append({'exp_name': exp_name, 'rf_name': rf_name})

{'exp_name': 'ACCESS-CM2_hist', 'rf_name': 'BirchLane'}
{'exp_name': 'ACCESS-CM2_hist', 'rf_name': 'Pinehaven'}
{'exp_name': 'ACCESS-CM2_ssp', 'rf_name': 'BirchLane'}
{'exp_name': 'ACCESS-CM2_ssp', 'rf_name': 'Pinehaven'}
{'exp_name': 'AWI-CM-1-1-MR_hist', 'rf_name': 'BirchLane'}
{'exp_name': 'AWI-CM-1-1-MR_hist', 'rf_name': 'Pinehaven'}
{'exp_name': 'AWI-CM-1-1-MR_ssp', 'rf_name': 'BirchLane'}
{'exp_name': 'AWI-CM-1-1-MR_ssp', 'rf_name': 'Pinehaven'}
{'exp_name': 'CNRM-CM6-1_hist', 'rf_name': 'BirchLane'}
{'exp_name': 'CNRM-CM6-1_hist', 'rf_name': 'Pinehaven'}
{'exp_name': 'CNRM-CM6-1_ssp', 'rf_name': 'BirchLane'}
{'exp_name': 'CNRM-CM6-1_ssp', 'rf_name': 'Pinehaven'}
{'exp_name': 'EC-Earth3_hist', 'rf_name': 'BirchLane'}
{'exp_name': 'EC-Earth3_hist', 'rf_name': 'Pinehaven'}
{'exp_name': 'EC-Earth3_ssp', 'rf_name': 'BirchLane'}
{'exp_name': 'EC-Earth3_ssp', 'rf_name': 'Pinehaven'}
{'exp_name': 'GFDL-ESM4_hist', 'rf_name': 'BirchLane'}
{'exp_name': 'GFDL-ESM4_hist', 'rf_name': 'Pineha

In [8]:
import re
import s3fs
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker import AutoML

def get_csv_from_s3(s3uri, file_name):
    parsed_url = urlparse(s3uri)
    bucket_name = parsed_url.netloc
    prefix = parsed_url.path[1:].strip("/")
    s3 = boto3.resource("s3")
    obj = s3.Object(bucket_name, "{}/{}".format(prefix, file_name))
    return obj.get()["Body"].read().decode("utf-8")

bucket_name = 'niwa-water-demand-modelling'
fs = s3fs.S3FileSystem(anon=False)
region = boto3.Session().region_name
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = "sagemaker/autopilot-water-demand-prediction"
role = get_execution_role()

rf_dict = {
    'Pinehaven': ['PCDUpperHutt'],
    'BirchLane': ['PCDStokesValley', 'PCDLowerHutt']
}

model_lst = ['CUR', 'ABI', 'ABJ', 'ABL', 'ABK']

In [9]:
def prepare_all_jobs(csv_files_rf, temp_extracts, csv_files_temp, completed_files):
    """Prepare all job combinations with better organization"""
    '''BirchLane_ACCESS-CM2_hist_PCDLowerHuttABI'''
    all_jobs = []
    
    for path in csv_files_rf:
        match = re.search(r'/([^/_]+(?: [^/_]+)*)_([^/_]+_[^/_]+)_adjusted', path)
        if match:
            rf_name = match.group(1).replace(' ', '')
            exp_name = match.group(2)
            scen_name = exp_name.split('_')[-1]
            if scen_name == 'hist':
                scen_name = 'historical'
            
            # Get job names for this location
            if rf_name in rf_dict:
                job_prefixes = rf_dict[rf_name]
                for job_prefix in job_prefixes:
                    for job_suffix in model_lst:
                        job_name = f'{job_prefix}{job_suffix}'
                        completed_file_key = f'{rf_name}_{exp_name}_{job_name}'
                        completed_file_found = [e for e in completed_files if completed_file_key in e]
                        if len(completed_file_found)>0:
                            print(f'{completed_file_key} found, skip job creation')
                            continue
                        all_jobs.append({
                            'file_data': (path, rf_name, exp_name, scen_name),
                            'job_name': job_name,
                            'priority': len(job_name)
                        })
    
    return all_jobs

In [10]:
csv_files_rf[0:10]

['WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_hist_adjusted_1960_2014.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_ssp126_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_ssp245_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_ssp370_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_ssp585_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Pinehaven_ACCESS-CM2_hist_adjusted_1960_2014.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Pinehaven_ACCESS-CM2_ssp126_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Pinehaven_ACCESS-CM2_ssp245_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Pinehaven_ACCESS-CM2_ssp370_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Pinehaven_ACCESS-CM2_ssp585_adjusted_2015_2100.csv']

In [13]:
# Job setting and input file filtering
instance_type = 'ml.c5.xlarge'
instance_count = 4
sagemaker_model_service_worker = '2'
csv_files_rf = [ 
    'WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_ssp370_adjusted_2015_2100.csv',
 'WWL/bias_adjusted_data/ACCESS-CM2/Birch Lane_ACCESS-CM2_ssp585_adjusted_2015_2100.csv'
]

In [ ]:
# Get completed files
completed_files = list_csv_files(bucket, prefix+'/cmip6-result')
completed_files = [e for e in completed_files if 'pred_data' in e]

# Extract exp_name and rf_name from each string
for path in csv_files_rf:
    match = re.search(r'/([^/_]+(?: [^/_]+)*)_([^/_]+_[^/_]+)_adjusted', path)
    if match:
        rf_name = match.group(1)
        rf_name = rf_name.replace(' ', '')
        exp_name = match.group(2)
        scen_name = exp_name.split('_')[-1]
        if scen_name == 'hist':
            scen_name = 'historical'
        
        # look for temp extracts info
        for i, x in enumerate(temp_extracts):
            exp_name_x = x['exp_name']
            rf_name_x = x['rf_name']
            if rf_name_x.lower() == rf_name.lower() and exp_name_x in exp_name:
                temp_key = csv_files_temp[i]
                print({'rf_name': rf_name, 'exp_name': exp_name, 'scen_name': scen_name})
                print(f'temp key_file: {temp_key}')
                break
        df_rf = read_csv_files_to_dataframes(bucket_name, [path])[0]
        df_rf.rename(columns={'time': 'Time', '0': 'Rainfall'}, inplace=True)
        df_rf = df_rf.set_index(pd.to_datetime(df_rf["Time"], format="%Y-%m-%d %H:%M:%S"))
        df_rf = df_rf.resample('h').sum(['Rainfall'])
        df_temp = read_csv_files_to_dataframes(bucket_name, [temp_key])[0]
        df_temp['Dry bulb degC'] = df_temp[scen_name] - 273
        df_temp = df_temp[['date', 'Dry bulb degC']]
        df_temp['date'] = pd.to_datetime(df_temp['date'], format="%Y-%m-%d %H:%M:%S")
        df_temp = df_temp.set_index('date')

        df_rf_1 = prep_rainfall(df_rf[['Rainfall']].reset_index())
        df = df_rf_1.set_index('Time').join(df_temp['Dry bulb degC'])
        df = df.reset_index()

        columns = [e for e in df.columns if e not in ["Time", "sin_hour", "cos_hour"]]
        test_data = df[columns] # Features
        test_file = f"{rf_name}_{exp_name}.csv"
        test_data.to_csv(test_file, index=False, header=False)
        test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix)
        print("Test data uploaded to: " + test_data_s3_path)

        # find suitable job names
        job_names = []
        job_prefixes = rf_dict[rf_name]
        for job_prefix in job_prefixes:
            for job_suffix in model_lst:
                job_name = f'{job_prefix}{job_suffix}'
                completed_file_key = f'{rf_name}_{exp_name}_{job_name}'
                completed_file_found = [e for e in completed_files if completed_file_key in e]
                if len(completed_file_found)>0:
                    print(f'{completed_file_key} found, skip job creation')
                    continue
                job_names.append(job_name)
        print("all job names: ", job_names)

        for job_name in job_names:
            print("running job: ", job_name)
            # This is the client we will use to interact with SageMaker Autopilot
            sm = boto3.Session().client(service_name="sagemaker", region_name=region)
            auto_ml_job_name = f"automl-{job_name}"
            print("AutoMLJobName: " + auto_ml_job_name)
            best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)["BestCandidate"]
            best_candidate_name = best_candidate["CandidateName"]
            
            print("\n")
            print("CandidateName: " + best_candidate_name)
            print(
                "FinalAutoMLJobObjectiveMetricName: "
                + best_candidate["FinalAutoMLJobObjectiveMetric"]["MetricName"]
            )
            print(
                "FinalAutoMLJobObjectiveMetricValue: "
                + str(best_candidate["FinalAutoMLJobObjectiveMetric"]["Value"])
            )
        
            automl = AutoML.attach(auto_ml_job_name=auto_ml_job_name)
            
            s3_transform_output_path = "s3://{}/{}/cmip6-results/".format(bucket, prefix)
            
            model_name = "{0}-model".format(best_candidate_name)
            
            model = automl.create_model(
                name=model_name,
                candidate=best_candidate,
            )
            
            output_path = s3_transform_output_path + best_candidate_name + "/"
            
            transformer = model.transformer(
                instance_count=instance_count,
                instance_type=instance_type,
                assemble_with="Line",
                strategy="SingleRecord",
                output_path=output_path,
                env={"SAGEMAKER_MODEL_SERVER_TIMEOUT": "100", "SAGEMAKER_MODEL_SERVER_WORKERS": sagemaker_model_service_worker},
            )
        
            transformer.transform(
                data=test_data_s3_path,
                split_type="Line",
                content_type="text/csv",
                wait=False,
                model_client_config={"InvocationsTimeoutInSeconds": 80, "InvocationsMaxRetries": 1},
            )
            
            print("Starting transform job {}".format(transformer._current_job_name))
        
            ## Wait for jobs to finish
            pending_complete = True
            batch_job_name = transformer._current_job_name
            
            while pending_complete:
                pending_complete = False
            
                description = sm.describe_transform_job(TransformJobName=batch_job_name)
                if description["TransformJobStatus"] not in ["Failed", "Completed"]:
                    pending_complete = True
            
                print("{} transform job is running.".format(batch_job_name))
                time.sleep(60)
            
            print("\nCompleted.")
            
            job_status = sm.describe_transform_job(TransformJobName=batch_job_name)["TransformJobStatus"]
            
            if job_status == "Completed":
                pred_csv = get_csv_from_s3(transformer.output_path, "{}.out".format(test_file))
                predictions = pd.read_csv(io.StringIO(pred_csv), header=None)
    
                df[job_name] = predictions
                df[job_name] = np.where(df[job_name] < 0, 0, df[job_name])
                pred_to_move = df.pop(job_name)
                df.insert(1, pred_to_move.name, pred_to_move)
        
            ### Upload the dataset to S3
            pred_file = f"{rf_name}_{exp_name}_{job_name}_pred_data.csv"
            df[['Time', job_name]].to_csv(f'cmip6/{pred_file}', index=False, header=True)
            pred_data_s3_path = session.upload_data(path=f'cmip6/{pred_file}', key_prefix=prefix + f"/cmip6-results")
            print("Full pred results uploaded to: " + pred_data_s3_path)


{'rf_name': 'BirchLane', 'exp_name': 'ACCESS-CM2_ssp370', 'scen_name': 'ssp370'}
temp key_file: WWL/cmip6/tas_ACCESS-CM2_ssp_BirchLane.csv


/tmp/ipykernel_3478/1009058355.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8008580206408932' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'API'] = k_api * df.at[i - 1, 'API'] + df.at[i - 1, 'Rainfall']
/tmp/ipykernel_3478/1009058355.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8008580206408932' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'SoilMoisture'] = min(


Test data uploaded to: s3://sagemaker-ap-southeast-2-851725470721/sagemaker/autopilot-water-demand-prediction/BirchLane_ACCESS-CM2_ssp370.csv
all job names:  ['PCDStokesValleyCUR', 'PCDStokesValleyABI', 'PCDStokesValleyABJ', 'PCDStokesValleyABL', 'PCDStokesValleyABK', 'PCDLowerHuttCUR', 'PCDLowerHuttABI', 'PCDLowerHuttABJ', 'PCDLowerHuttABL', 'PCDLowerHuttABK']
running job:  PCDStokesValleyCUR
AutoMLJobName: automl-PCDStokesValleyCUR


CandidateName: automl-PCDStokesValleyCURzNNb5wX-001-0bfb2f08
FinalAutoMLJobObjectiveMetricName: validation:mse
FinalAutoMLJobObjectiveMetricValue: 0.0


[06/30/25 10:08:42] INFO     Creating model with name:                                              ]8;id=566214;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=593025;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\
                             automl-PCDStokesValleyCURzNNb5wX-001-0bfb2f08-model                                   

[06/30/25 10:08:43] WARNING  Using already existing model:                                          ]8;id=202795;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=880789;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4105\4105]8;;\
                             automl-PCDStokesValleyCURzNNb5wX-001-0bfb2f08-model                                   

                    INFO     Creating transform job with name:                                      ]8;id=79810;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=920093;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#3951\3951]8;;\
                             automl-PCDStokesValleyCURzNNb5wX-001-0b-2025-06-30-10-08-43-911                       

Starting transform job automl-PCDStokesValleyCURzNNb5wX-001-0b-2025-06-30-10-08-43-911
automl-PCDStokesValleyCURzNNb5wX-001-0b-2025-06-30-10-08-43-911 transform job is running.


In [ ]:
df.dropna()

,Time,PCDLowerHuttABK,PCDLowerHuttABL,PCDLowerHuttABJ,PCDStokesValleyABK,PCDStokesValleyABL,PCDStokesValleyABJ,Rainfall,hour,wday,...,Rain_L3HR,Rain_L6HR,Rain_L12HR,Rain_L24HR,Rain_L48HR,sin_hour,cos_hour,API,SoilMoisture,Dry bulb degC
1,2015-01-01 01:00:00,0.005005,0.013659,0.011536,0.018342,0.022497,0.041341,0.0,1,3,...,0.0,0.0,0.0,0.0,0.0,0.258819,0.965926,0.000000,0.000000,22.26290
2,2015-01-01 02:00:00,0.004212,0.013311,0.010837,0.017125,0.022497,0.041341,0.0,2,3,...,0.0,0.0,0.0,0.0,0.0,0.500000,0.866025,0.000000,0.000000,22.31200
3,2015-01-01 03:00:00,0.004095,0.009412,0.010664,0.014813,0.021325,0.041235,0.0,3,3,...,0.0,0.0,0.0,0.0,0.0,0.707107,0.707107,0.000000,0.000000,21.59845
4,2015-01-01 04:00:00,0.003834,0.008721,0.010180,0.014813,0.021325,0.038265,0.0,4,3,...,0.0,0.0,0.0,0.0,0.0,0.866025,0.500000,0.000000,0.000000,20.61478
5,2015-01-01 05:00:00,0.003743,0.007612,0.007913,0.014862,0.021325,0.034630,0.0,5,3,...,0.0,0.0,0.0,0.0,0.0,0.965926,0.258819,0.000000,0.000000,20.09120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745100,2099-12-31 20:00:00,0.000092,0.001729,0.000288,0.000000,0.000000,0.000000,0.0,20,3,...,0.0,0.0,0.0,0.0,0.0,-0.866025,0.500000,0.000224,0.403105,22.48352
745101,2099-12-31 21:00:00,0.000090,0.001848,0.000288,0.000000,0.000000,0.000000,0.0,21,3,...,0.0,0.0,0.0,0.0,0.0,-0.707107,0.707107,0.000191,0.382950,23.51288
745102,2099-12-31 22:00:00,0.000090,0.001848,0.000288,0.000000,0.000000,0.000000,0.0,22,3,...,0.0,0.0,0.0,0.0,0.0,-0.500000,0.866025,0.000162,0.363802,24.46170
745103,2099-12-31 23:00:00,0.000090,0.001812,0.000288,0.000000,0.000000,0.000000,0.0,23,3,...,0.0,0.0,0.0,0.0,0.0,-0.258819,0.965926,0.000138,0.345612,25.08408
